In [1]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
from sqlalchemy import Column, Integer, String

class UserModel(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    username = Column(String)
    
class RoleModel(Base):
    __tablename__ = 'role'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class UserRoleModel(Base):
    __tablename__ = 'user_role'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer)
    role_id = Column(Integer)
    
Base.metadata.create_all(engine)

2018-03-08 03:06:34,059 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-08 03:06:34,060 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 03:06:34,062 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-08 03:06:34,063 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 03:06:34,064 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-03-08 03:06:34,065 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 03:06:34,068 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("role")
2018-03-08 03:06:34,069 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 03:06:34,070 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user_role")
2018-03-08 03:06:34,070 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 03:06:34,072 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	username VARCHAR, 
	PRIMARY KEY (id)
)


2018-03-08 03:06:34,072 INFO sqlalchemy.engin

In [3]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [4]:
session = Session()

user1 = UserModel(username="u1")
user2 = UserModel(username="u2")

role1 = RoleModel(name="r1")
role2 = RoleModel(name="r1")

session.add_all([user1, user2, role1, role2])
session.flush()

session.add_all([
    UserRoleModel(user_id = user1.id, role_id = role1.id), 
    UserRoleModel(user_id = user2.id, role_id = role2.id)
])

session.commit()
session.query(UserRoleModel).filter().all()

2018-03-08 03:06:34,109 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 03:06:34,113 INFO sqlalchemy.engine.base.Engine INSERT INTO role (name) VALUES (?)
2018-03-08 03:06:34,113 INFO sqlalchemy.engine.base.Engine ('r1',)
2018-03-08 03:06:34,115 INFO sqlalchemy.engine.base.Engine INSERT INTO role (name) VALUES (?)
2018-03-08 03:06:34,116 INFO sqlalchemy.engine.base.Engine ('r1',)
2018-03-08 03:06:34,117 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username) VALUES (?)
2018-03-08 03:06:34,119 INFO sqlalchemy.engine.base.Engine ('u1',)
2018-03-08 03:06:34,120 INFO sqlalchemy.engine.base.Engine INSERT INTO users (username) VALUES (?)
2018-03-08 03:06:34,121 INFO sqlalchemy.engine.base.Engine ('u2',)
2018-03-08 03:06:34,124 INFO sqlalchemy.engine.base.Engine INSERT INTO user_role (user_id, role_id) VALUES (?, ?)
2018-03-08 03:06:34,126 INFO sqlalchemy.engine.base.Engine (1, 1)
2018-03-08 03:06:34,127 INFO sqlalchemy.engine.base.Engine INSERT INTO user_role (user_id,

In [5]:
from graphene_sqlalchemy import SQLAlchemyObjectType
import graphene

class User(SQLAlchemyObjectType):
    class Meta:
        model = UserModel
        
class Query(graphene.ObjectType):
    users = graphene.List(User)

    def resolve_users(self, info):
        query = User.get_query(info)  # SQLAlchemy query
        return query.all()

schema = graphene.Schema(query=Query)

In [6]:
result = schema.execute(
    '''
        query {
          users {
            username,
          }
        }
    ''', 
    context_value={'session': session}
).data

import json
def dit(ordered_dict):  
    # ordered_dict => normal dict
    return json.loads(json.dumps(ordered_dict))

print(result)
dit(result)

2018-03-08 03:06:34,384 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.username AS users_username 
FROM users
2018-03-08 03:06:34,384 INFO sqlalchemy.engine.base.Engine ()
OrderedDict([('users', [OrderedDict([('username', 'u1')]), OrderedDict([('username', 'u2')])])])


{'users': [{'username': 'u1'}, {'username': 'u2'}]}